In [ ]:
import pandas as pd
from pyspark.mllib.feature import Word2Vec, Word2VecModel
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
import h5py
import os
import sys
import logging
from sklearn.model_selection import ParameterGrid
import shutil

In [ ]:
partitions = os.cpu_count() - 2

In [ ]:
# https://stackoverflow.com/questions/40208420/how-to-find-hdf5-file-groups-keys-within-python
with h5py.File('binarized.hdf') as f:
    print(f.keys())

In [ ]:
sc = SparkContext('local[{cpus}]'.format(cpus=partitions), 'word2vec')
spark = SparkSession(sc)

In [ ]:
# COLUMNS
LIKED = 'Liked'
MOVIE_ID = 'movieId'
USER_ID = 'userId'
TIMESTAMP = 'Timestamp'
TITLE = 'title'
GENRE = 'genres'

In [ ]:
def transform_df(_df):
    _df = _df.drop([TIMESTAMP], axis=1)
    _df[MOVIE_ID] = _df.index.get_level_values(MOVIE_ID).astype(str)
    return _df
    

In [ ]:
df_movies = pd.read_csv('ml-20m/movies.csv', index_col=MOVIE_ID)

In [ ]:
df_trg = pd.read_hdf('binarized.hdf', key='trg')
df_trg = df_trg[df_trg[LIKED] == 1]
df_trg = df_trg.head(50000) # todo comment out for production
df_trg = transform_df(df_trg)

df_val = transform_df(pd.read_hdf('binarized.hdf', key='val'))

In [ ]:
# PARAMETERS
# The most ratings any user has had
df_trg_gb = df_trg.groupby([USER_ID])
dict_groups_trg = {k: list(v[MOVIE_ID]) for k, v in df_trg_gb}
WINDOW_SIZE = df_trg_gb[LIKED].count().max()

In [ ]:
df_val_gb = df_val.groupby([USER_ID])
dict_groups_val = {k: list(v[MOVIE_ID]) for k, v in df_val_gb}

In [ ]:
LEARNING_RATE = 'learning_rate'
VECTOR_SIZE = 'vector_size'
MIN_COUNT = 'min_count'
param_grid = ParameterGrid({
    LEARNING_RATE: [0.025, 0.01, 0.05],
    VECTOR_SIZE: [64, 128],
    MIN_COUNT: [5, 10, 20]
})

In [ ]:
list(param_grid)

In [ ]:
document = sc.parallelize(dict_groups_trg.values(), partitions)

In [ ]:
list_performance = list()

In [ ]:
for params in param_grid:
    logging.debug('Params: {params}'.format(params=params))
    logging.debug('Start Train: {ts}'.format(ts=pd.Timestamp('now')))
    
    # Constant
    word2vec = Word2Vec()
    word2vec.setNumPartitions(partitions)
    word2vec.setWindowSize(WINDOW_SIZE)
    
    # Grid
    word2vec.setLearningRate(params[LEARNING_RATE])
    word2vec.setVectorSize(params[VECTOR_SIZE])
    word2vec.setMinCount(params[MIN_COUNT])
    
    # Fit and save
    model = word2vec.fit(document)
    logging.debug('Stop Train: {ts}'.format(ts=pd.Timestamp('now')))
    outdir = 'w2v_model_pt_{partitions}_mc_{min_count}_lr_{lr}_vs_{vs}.sparkmodel'.format(
        partitions=partitions, min_count=params[MIN_COUNT], lr=params[LEARNING_RATE],
        vs=params[VECTOR_SIZE]
    )
    if os.path.isdir(outdir):
        shutil.rmtree(outdir)
    model.save(sc, outdir)
    
    break

In [ ]:
x = model.transform('1234')

In [ ]:
x

In [ ]:
model.transform(['1234', '1234'])

In [ ]:
dict_groups_val[1]

In [ ]:
model.transfor

In [ ]:
def show_synonyms(search_str, num_synonyms):
    synonym_list = list()
    movie_index = df_movies[df_movies[TITLE].str.match(search_str)]
    print(movie_index)
    for mi in movie_index.index:
        synonym_list.extend([(i, df_movies.loc[int(i[0])][TITLE]) for i in 
                             list(model.findSynonyms(str(mi), num_synonyms))])
    return synonym_list

In [ ]:
# word2vec2 = Word2VecModel.load(sc, 'trained_wor2vec_pyspark.sparkmodel')

In [ ]:
show_synonyms('.*Matrix.*', 5)

In [ ]:
show_synonyms('.*Private Ryan.*', 5)